In [16]:
import sys
import os
import time
sys.path.append(os.path.abspath(os.path.dirname(os.getcwd())))
sys.path.append(os.path.abspath(os.getcwd()))

import casadi as cd
from Envs.robot_arm import EFFECTOR_env_mj, End_Effector_model
from Solvers.OCsolver import  ocsolver_v4
import numpy as np
from matplotlib import pyplot as plt

from utils.Visualize_mj import arm_visualizer_mj_v1
from scipy.spatial.transform import Rotation as R
import mujoco

In [17]:
filepath = os.path.join(os.path.abspath(os.path.dirname(os.getcwd())),
                    'mujoco_arm', 'franka_emika_panda',
                    'scene_pos.xml')
print('path', filepath)

path /home/zxxie/ASU_research/safe_alignment/code/Safety-Alignment-with-Human-Directional-Feedback/mujoco_arm/franka_emika_panda/scene_pos.xml


In [18]:
model = mujoco.MjModel.from_xml_path(filepath)
data=mujoco.MjData(model)
viewer=mujoco.viewer.launch(model, data)